# Maven Power Outage Challenge

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium 
import openpyxl

## Step 0. Initial exploration of the data - it is messy!

In [22]:
# disturbances = pd.read_excel()

disturbances = pd.read_excel('DOE_Electric_Disturbance_Events.xlsx', engine='openpyxl')
disturbances

,Table B.2.,"Major Disturbances and Unusual Occurrences, 2002",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Date,NERC Region,Time,Area,Type of Disturbance,Loss (megawatts),Number of Customers Affected,Restoration Time
2,January,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2002-01-30 00:00:00,SPP,06:00:00,Oklahoma,Ice Storm,500,1881134,2002-02-07 12:00:00
4,None,NaN,None,NaN,NaN,NaN,NaN,None
5,2002-01-29 00:00:00,SPP,Evening,Metropolitan Kansas City Area,Ice Storm,500-600,270000,NaN
6,2002-01-30 00:00:00,SPP,16:00:00,Missouri,Ice Storm,210,95000,2002-02-10 21:00:00
7,February,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2002-02-27 00:00:00,WSCC,10:48:00,California,Interruption of Firm Load,300,255000,2002-02-27 11:35:00
9,March,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
disturbances.columns

Index(['Table B.2.', 'Major Disturbances and Unusual Occurrences, 2002',
       'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6',
       'Unnamed: 7'],
      dtype='object')

Initial notes about data:
1. the column headers seem to be located in row 1
2. date formats are inconsistent
3. I will need to check what NERC Region means
4. What does area refer to? Sometimes it is an entire state while other times it is a more specific part of a state.
5. lots of NaN that needs to be addressed
6. some columns have inconsistent data located in them - perhaps these rows need to be removed

In [21]:
#What is the shape of the data:

disturbances.shape

(41, 8)

## Step 1. Creating a rough data cleaning plan in pseudocode 

After taking an initial look at the data I know I want to use the python package openpyxl to clean/ wrangle the data programmatically. Here is what I want to accomplish:
0. Load workbook so that we can read the data from the excel sheet with openpyxl
1. Initialise an empty pandas DataFrame. Define the column names: ["Date", "NERC Region", "Time", "Area Affected",	"Type of Disturbance", "Loss (megawatts)",	"Number of Customers Affected", "Restoration Date/Time"]
2. for sheets 2002 - 2010 we have columns A-H (in these sheets date and time of restoration are in one column called "Restoration Date/Time" which changes to two columns in 2011: "Date of Restoration", "Time of Restoration"):
    a. Select one sheet within the excel spreadsheet. loop rough the sheet line by line
    b. If row starts with "Table", remove row (or skip row)
    c. If row starts with a month or the word "Date", skip row
    d. else, append row to the DataFrame
2. for 2011-2014 we have columns A-I (look at data description to find out the new structure of the timing record). In 2011 "Type of disturbance" changes to "Event type".
3. for sheets 2015 - 2023 we have columns A-K: two columns have been added: "Month" (which replaces the row markers within the sheets, and "Alert Criteria"
3. Repeat step 2 for all sheets in the file
4. Check the output, start proper exploratory data analysis

In [19]:
#Step 0: Exploring openpyxl

# This is an example of how to return data from a predefined sheet
print(excel_workbook["2003"]["F7"].value) 

1000


In [25]:
#slicing the data - getting all cells from a column. I could also use iter_rows or iter_cols
print(excel_workbook["2015"]["A"])

[<Cell '2015'.A1>, <Cell '2015'.A2>, <Cell '2015'.A3>, <Cell '2015'.A4>, <Cell '2015'.A5>, <Cell '2015'.A6>, <Cell '2015'.A7>, <Cell '2015'.A8>, <Cell '2015'.A9>, <Cell '2015'.A10>, <Cell '2015'.A11>, <Cell '2015'.A12>, <Cell '2015'.A13>, <Cell '2015'.A14>, <Cell '2015'.A15>, <Cell '2015'.A16>, <Cell '2015'.A17>, <Cell '2015'.A18>, <Cell '2015'.A19>, <Cell '2015'.A20>, <Cell '2015'.A21>, <Cell '2015'.A22>, <Cell '2015'.A23>, <Cell '2015'.A24>, <Cell '2015'.A25>, <Cell '2015'.A26>, <Cell '2015'.A27>, <Cell '2015'.A28>, <Cell '2015'.A29>, <Cell '2015'.A30>, <Cell '2015'.A31>, <Cell '2015'.A32>, <Cell '2015'.A33>, <Cell '2015'.A34>, <Cell '2015'.A35>, <Cell '2015'.A36>, <Cell '2015'.A37>, <Cell '2015'.A38>, <Cell '2015'.A39>, <Cell '2015'.A40>, <Cell '2015'.A41>, <Cell '2015'.A42>, <Cell '2015'.A43>, <Cell '2015'.A44>, <Cell '2015'.A45>, <Cell '2015'.A46>, <Cell '2015'.A47>, <Cell '2015'.A48>, <Cell '2015'.A49>, <Cell '2015'.A50>, <Cell '2015'.A51>, <Cell '2015'.A52>, <Cell '2015'.A53>, <

In [27]:
for value in excel_workbook["2015"].iter_rows(min_row=1, max_row=2, min_col=1, max_col=3, values_only=True):
    print(value)


('OE-417 Electric Emergency and Disturbance Report - Calendar Year 2015', None, None)
('Month', 'Date Event Began', 'Time Event Began')


In [34]:
for row in excel_workbook["2015"].rows:
    print(row)

NameError: name 'cell' is not defined

In [11]:
#Step 1: Load workbook so that we can read the data from the excel sheet with openpyxl. Retrieve the sheet names.

# FYI There are additional reading options to keep in mind: read_only loads a spreadsheet in read-only mode allowing you to open very large Excel files.
# data_only ignores loading formulas and instead loads only the resulting values.

from openpyxl import load_workbook
excel_workbook = load_workbook(filename='DOE_Electric_Disturbance_Events.xlsx')
excel_sheetnames = excel_workbook.sheetnames
sheet = excel_workbook.active

1000
